In [1]:
import gzip

def readGz(f):
    for l in gzip.open(f):
        yield eval(l)

In [2]:
from collections import defaultdict
import numpy as np
data = []
count = 0;

for l in readGz("train.json.gz"):
    data.append(l)

trainData, validData = np.array_split(data, 2)

print(len(trainData))
print(len(validData))
print(trainData[0])

100000
100000
{'rating': 5.0, 'businessID': 'B408037852', 'reviewText': "This is where i go to shop for gifts from my mom. She loves this stuff. Cna't get enough. I like that you can customize the items. Store is well alid out and shoppable.", 'userID': 'U093387342', 'reviewTime': 'Mar 24, 2013', 'categories': ["Women's Clothing Store", 'Fashion Accessories Store', 'Shoe Store'], 'reviewHash': 'R471510664', 'unixReviewTime': 1364143460}


In [3]:
businessCount = defaultdict(int)
visitedBusinesses = defaultdict(int)
users = defaultdict(int)
totalVisits = 0;

for d in trainData:
    user, business = d['userID'], d['businessID']
    businessCount[business] += 1
    totalVisits += 1
    visitedBusinesses[user+'-'+business] += 1
    users[user] += 1

print(len(businessCount))
print(len(users))
print(len(visitedBusinesses))

mostPopular = [(businessCount[x], x) for x in businessCount]
mostPopular.sort()
mostPopular.reverse()

return1 = set()
count = 0
for ic, i in mostPopular:
  count += ic
  return1.add(i)
  if count > totalVisits/2: break

20490
18052
100000


In [4]:
businessList = []
usersList = []

'''for c in businessCount:
    businessList.append(c)
for u in users:
    usersList.append(u)''' #Not taking the pairs from training data, taking them from validation data

vbusinessCount = defaultdict(int)
vvisitedBusinesses = defaultdict(int)
vusers = defaultdict(int)
vtotalVisits = 0;
vnotvisited = defaultdict(int)

for d in validData:
    user, business = d['userID'], d['businessID']
    vbusinessCount[business] += 1
    vtotalVisits += 1
    vvisitedBusinesses[user+'-'+business] += 1
    vusers[user] += 1
    
print(len(vvisitedBusinesses))
print(len(vbusinessCount))
print(len(vusers))

for c in vbusinessCount:
    businessList.append(c)
for u in vusers:
    usersList.append(u)

import random

while(len(vnotvisited)<100000):
    pair = random.choice(usersList)+'-'+random.choice(businessList)
    while(pair in vvisitedBusinesses):
        pair = random.choice(usersList)+'-'+random.choice(businessList)
    vnotvisited[pair] += 1

print(len(vnotvisited))

100000
20521
18079
100000


In [5]:
wrongPred = 0;
correctPred = 0;

def baselinePredict(userBusinessPair, visited):
    global wrongPred
    global correctPred
    global return1
    print(wrongPred)
    for p in userBusinessPair:
        u,b = p.strip().split('-')
        if((b in return1 and visited) or (b not in return1 and not visited)):
            correctPred += 1
        else:
            wrongPred += 1

baselinePredict(vvisitedBusinesses, True)
baselinePredict(vnotvisited, False)

#print(wrongPred)
#print(correctPred)
print(wrongPred/(wrongPred+correctPred)*100)

0
55784
37.6445


In [6]:
mostPopular = [(businessCount[x], x) for x in businessCount]
mostPopular.sort()
mostPopular.reverse()

return1 = set()
count = 0
for ic, i in mostPopular:
  count += ic
  return1.add(i)
  if count > 0.65*totalVisits: break

wrongPred = 0
correctPred = 0
baselinePredict(vvisitedBusinesses, True)
baselinePredict(vnotvisited, False)

print(wrongPred/(wrongPred+correctPred)*100)

0
43479
37.835


In [24]:
visitedCategory = defaultdict(set)
print(trainData[0])

{'rating': 5.0, 'businessID': 'B408037852', 'reviewText': "This is where i go to shop for gifts from my mom. She loves this stuff. Cna't get enough. I like that you can customize the items. Store is well alid out and shoppable.", 'userID': 'U093387342', 'reviewTime': 'Mar 24, 2013', 'categories': ["Women's Clothing Store", 'Fashion Accessories Store', 'Shoe Store'], 'reviewHash': 'R471510664', 'unixReviewTime': 1364143460}


In [25]:
for d in trainData:
    for c in d['categories']:
        visitedCategory[d['userID']].add(c)
        
print("done")
print(len(visitedCategory))

done
18052


In [26]:
bCategory = defaultdict(set)
print(validData[5])

{'rating': 4.0, 'businessID': 'B903181568', 'reviewText': "This is a great spot.  You could eat really cheaply or more... Good local hangout, and the bartenders are great.  Good mac and cheese....good salads, good 'bad food', and great 'quarter pounder' cookies!", 'userID': 'U455837535', 'reviewTime': 'May 3, 2013', 'categories': ['American Restaurant'], 'reviewHash': 'R506914583', 'unixReviewTime': 1367595704, 'categoryID': 0}


In [27]:
for d in validData:
    for c in d['categories']:
        bCategory[d['businessID']].add(c)
print("done")
print(len(bCategory))

done
20521


In [28]:
wPred = 0
cPred = 0

def categoryVisitedPred(ubPairs, visited):
    global wPred
    global cPred
    for p in ubPairs:
        u,b = p.strip().split('-')
        match = False;
        for c in bCategory[b]:
            for uc in visitedCategory[u]:
                if(c==uc):
                    match = True
                    break
            if(match):
                break
        if((match and visited) or (not match and not visited)):
            cPred += 1
        else:
            wPred += 1

categoryVisitedPred(vvisitedBusinesses, True)
categoryVisitedPred(vnotvisited, False)

print(wPred)
print(cPred)
print(wPred/(wPred+cPred))

69473
130527
0.347365


In [22]:
visitedCategory = defaultdict(set)
bCategory = defaultdict(set)

for d in data:
    #print(d)
    for c in d['categories']:
        visitedCategory[d['userID']].add(c)
        bCategory[d['businessID']].add(c)
print("done")
print(len(visitedCategory))
print(len(bCategory))

done
18793
21321


In [29]:
def categoryVisitedPredFile(inputFile, outputFile):
    predictions = open(outputFile, 'w')
    for l in open(inputFile):
        if l.startswith("userID"):
            predictions.write(l)
            continue
        u,b = l.strip().split('-')
        match = False;
        for c in bCategory[b]:
            for uc in visitedCategory[u]:
                if(c==uc):
                    #print(c)
                    match = True
                    break
            if(match):
                break
        if(match):
            predictions.write(u + '-' + b + ",1\n")
        else:
            predictions.write(u + '-' + b + ",0\n")

categoryVisitedPredFile ("pairs_Visit.txt", "predictions_Visit.txt")    

print("done")

done
